In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
%%bash
# Logistics #2: install the transformers package, create a folder, download the dataset and a patch
pip -q install transformers
pip -q install datasets
pip -q install tqdm
pip -q install sentencepiece 

# remove the directory if necessary
# rm -rf "/content/gdrive/MyDrive/6864_project/"

#mkdir "/content/gdrive/MyDrive/6864_project/"
cd "/content/gdrive/MyDrive/6864_project/"

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
import torch.nn as nn

In [ ]:
!ls gdrive/MyDrive/6864_project/data/infertility



all_posts_with_comment_topics.csv  posts.csv  processed_posts.csv


In [ ]:
data = pd.read_csv("gdrive/MyDrive/6864_project/data/infertility/processed_posts.csv")

In [ ]:
data_ttc = pd.read_csv("gdrive/MyDrive/6864_project/data/ttcafterloss/processed_posts.csv")
dfs = [data, data_ttc]
data = pd.concat(dfs)

In [ ]:
topic = pd.read_csv("gdrive/MyDrive/6864_project/data/10_all_posts_with_comment_topics.csv")

In [ ]:
data.head()

,Unnamed: 0,id,created_utc,author,author_fullname,author_flair_text,url,title,selftext,upvote_ratio,score,num_comments,data_split,processed_text,processed_title
0,0,2t72mf,1.421865e+09,nhmejia,t2_ivw2a,Hard to get pregnant after a hysto.,https://www.reddit.com/r/infertility/comments/...,*TRIGGER WARNING* Not that funny...,I've been doing really well staying away from ...,0.80,8,25,val,"[['I', 'be', 'do', 'really', 'well', 'stay', '...","[['trigger', 'warn', 'not', 'that', 'funny']]"
1,1,2t6u8l,1.421862e+09,septicidal,t2_6h9ib,PCOS,https://www.reddit.com/r/infertility/comments/...,Update on planning for a new cycle,I know a bunch of people saw my update in the ...,1.00,7,8,train,"[['I', 'know', 'a', 'bunch', 'of', 'people', '...","[['update', 'on', 'plan', 'for', 'a', 'new', '..."
2,3,2t57np,1.421820e+09,PoisonRose23,t2_j2zbu,NaN,https://www.reddit.com/r/infertility/comments/...,Decision about fermara! I need advice!!!,Long post warning: So I have a decision to mak...,1.00,1,6,train,"[['long', 'post', 'warn', 'so', 'I', 'have', '...","[['decision', 'about', 'fermara', 'I', 'need',..."
3,5,2t45rt,1.421801e+09,Pamzella,t2_aifrd,41 MFI & DOR 1MC mult IUIs,https://www.reddit.com/r/infertility/comments/...,"""Shared risk"" clinics","Please educate me, someone asked me about this...",1.00,7,17,train,"[['please', 'educate', 'I', 'someone', 'ask', ...","[['share', 'risk', 'clinic']]"
4,6,2t3x6r,1.421797e+09,fumbellina,t2_iibv6,NaN,https://www.reddit.com/r/infertility/comments/...,FET anxiety,"My FET is scheduled for next week, and I'm fee...",0.81,3,6,train,"[['my', 'fet', 'be', 'schedule', 'for', 'next'...","[['fet', 'anxiety']]"


In [ ]:
data = data.merge(topic[["id","max_mean_topic","mode_max_topic"]], on='id')

In [ ]:
data = data[~data.max_mean_topic.isna()]

In [ ]:
set(data.max_mean_topic)

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0}

In [ ]:
#Handling long data inputs

#split into chunks of 200 words with 50 words overlapped
def get_split(text1):
  
  text1 = "".join(text1.split("]"))
  text1 = "".join(text1.split("["))

  text1 = " ".join(text1.split(","))

  text1 = text1.replace("'", "")

  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:200]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_parcial))
  return l_total

data = data[data.processed_text.str.len() > 10]
data["processed_text"] = data["processed_title"] + "," + data["processed_text"]
data["text_split"] = data["processed_text"].apply(get_split)
data.head()

,Unnamed: 0,id,created_utc,author,author_fullname,author_flair_text,url,title,selftext,upvote_ratio,score,num_comments,data_split,processed_text,processed_title,max_mean_topic,mode_max_topic,text_split
0,0,2t72mf,1.421865e+09,nhmejia,t2_ivw2a,Hard to get pregnant after a hysto.,https://www.reddit.com/r/infertility/comments/...,*TRIGGER WARNING* Not that funny...,I've been doing really well staying away from ...,0.80,8,25,val,"[['trigger', 'warn', 'not', 'that', 'funny']],...","[['trigger', 'warn', 'not', 'that', 'funny']]",4.0,4.0,[trigger warn not that funny I be do really we...
1,1,2t6u8l,1.421862e+09,septicidal,t2_6h9ib,PCOS,https://www.reddit.com/r/infertility/comments/...,Update on planning for a new cycle,I know a bunch of people saw my update in the ...,1.00,7,8,train,"[['update', 'on', 'plan', 'for', 'a', 'new', '...","[['update', 'on', 'plan', 'for', 'a', 'new', '...",4.0,4.0,[update on plan for a new cycle I know a bunch...
2,3,2t57np,1.421820e+09,PoisonRose23,t2_j2zbu,NaN,https://www.reddit.com/r/infertility/comments/...,Decision about fermara! I need advice!!!,Long post warning: So I have a decision to mak...,1.00,1,6,train,"[['decision', 'about', 'fermara', 'I', 'need',...","[['decision', 'about', 'fermara', 'I', 'need',...",6.0,6.0,[decision about fermara I need advice long pos...
3,5,2t45rt,1.421801e+09,Pamzella,t2_aifrd,41 MFI & DOR 1MC mult IUIs,https://www.reddit.com/r/infertility/comments/...,"""Shared risk"" clinics","Please educate me, someone asked me about this...",1.00,7,17,train,"[['share', 'risk', 'clinic']],[['please', 'edu...","[['share', 'risk', 'clinic']]",6.0,6.0,[share risk clinic please educate I someone as...
4,6,2t3x6r,1.421797e+09,fumbellina,t2_iibv6,NaN,https://www.reddit.com/r/infertility/comments/...,FET anxiety,"My FET is scheduled for next week, and I'm fee...",0.81,3,6,train,"[['fet', 'anxiety']],[['my', 'fet', 'be', 'sch...","[['fet', 'anxiety']]",6.0,6.0,[fet anxiety my fet be schedule for next week ...


In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-cased')

In [ ]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
bert = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# max_len = 0

# # For every sentence...
# for post in data.text_split:
#   for chunk in post:
#     input_ids = tokenizer.encode(chunk, add_special_tokens=True)
#     max_len = max(max_len, len(input_ids))

# print('Max sentence length: ', max_len)


In [ ]:
data["input_ids"] = data["text_split"].apply((lambda x: [tokenizer.encode_plus(chunk, add_special_tokens=True, max_length=377, padding='max_length', return_attention_mask=True,return_tensors='pt')['input_ids'] for chunk in x]))

In [ ]:
data["attention_mask"] = data["text_split"].apply((lambda x: [tokenizer.encode_plus(chunk, add_special_tokens=True, max_length= 377, padding="max_length", return_attention_mask=True,return_tensors='pt')['attention_mask'] for chunk in x]))

In [ ]:
data.head()

,Unnamed: 0,id,created_utc,author,author_fullname,author_flair_text,url,title,selftext,upvote_ratio,score,num_comments,data_split,processed_text,processed_title,max_mean_topic,mode_max_topic,text_split,input_ids,attention_mask
0,0,2t72mf,1.421865e+09,nhmejia,t2_ivw2a,Hard to get pregnant after a hysto.,https://www.reddit.com/r/infertility/comments/...,*TRIGGER WARNING* Not that funny...,I've been doing really well staying away from ...,0.80,8,25,val,"[['trigger', 'warn', 'not', 'that', 'funny']],...","[['trigger', 'warn', 'not', 'that', 'funny']]",4.0,4.0,[trigger warn not that funny I be do really we...,"[[[tensor(101), tensor(9887), tensor(11857), t...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."
1,1,2t6u8l,1.421862e+09,septicidal,t2_6h9ib,PCOS,https://www.reddit.com/r/infertility/comments/...,Update on planning for a new cycle,I know a bunch of people saw my update in the ...,1.00,7,8,train,"[['update', 'on', 'plan', 'for', 'a', 'new', '...","[['update', 'on', 'plan', 'for', 'a', 'new', '...",4.0,4.0,[update on plan for a new cycle I know a bunch...,"[[[tensor(101), tensor(11984), tensor(1113), t...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."
2,3,2t57np,1.421820e+09,PoisonRose23,t2_j2zbu,NaN,https://www.reddit.com/r/infertility/comments/...,Decision about fermara! I need advice!!!,Long post warning: So I have a decision to mak...,1.00,1,6,train,"[['decision', 'about', 'fermara', 'I', 'need',...","[['decision', 'about', 'fermara', 'I', 'need',...",6.0,6.0,[decision about fermara I need advice long pos...,"[[[tensor(101), tensor(2383), tensor(1164), te...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."
3,5,2t45rt,1.421801e+09,Pamzella,t2_aifrd,41 MFI & DOR 1MC mult IUIs,https://www.reddit.com/r/infertility/comments/...,"""Shared risk"" clinics","Please educate me, someone asked me about this...",1.00,7,17,train,"[['share', 'risk', 'clinic']],[['please', 'edu...","[['share', 'risk', 'clinic']]",6.0,6.0,[share risk clinic please educate I someone as...,"[[[tensor(101), tensor(2934), tensor(3187), te...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."
4,6,2t3x6r,1.421797e+09,fumbellina,t2_iibv6,NaN,https://www.reddit.com/r/infertility/comments/...,FET anxiety,"My FET is scheduled for next week, and I'm fee...",0.81,3,6,train,"[['fet', 'anxiety']],[['my', 'fet', 'be', 'sch...","[['fet', 'anxiety']]",6.0,6.0,[fet anxiety my fet be schedule for next week ...,"[[[tensor(101), tensor(175), tensor(2105), ten...","[[[tensor(1), tensor(1), tensor(1), tensor(1),..."


In [ ]:
def get_bin(x):
  if x < 4:
    return 0
  if x < 8:
    return 1
  if x < 11:
    return 2
  if x < 16:
    return 3
  if x < 25:
    return 4
  return 5

In [ ]:
data["comments_bin"] = data["num_comments"].apply(lambda x: get_bin(x))

In [297]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, df):
        'Initialization'
        self.df = df

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.df)

  def __getitem__(self, index):

        # Load data and get label
        row = self.df.iloc[index]
        input_ids = torch.cat([x.float() for x in row.input_ids], dim=0).to(torch.int64)
        attention_mask = torch.cat([x.float() for x in row.attention_mask], dim=0)
        #label = torch.tensor(row.num_comments).to(torch.float)
        label = torch.tensor(row.comments_bin).to(torch.long)
        #label = torch.tensor(row.upvote_ratio).to(torch.float)
        #label = torch.tensor(row.mode_max_topic).to(torch.long)
        id = row.id
        sample = {"input_ids":input_ids, "attention_mask":attention_mask, "label":label, "id":id}
        #print(sample["label"], row.num_comments)
        return sample


In [298]:
#HIDDEN_DIM = 16384
HIDDEN_DIM = 40

#OUTPUT_DIM = 10
OUTPUT_DIM = 6

BATCH_SIZE = 1

In [299]:
def stack_vals(batch,key):
  lst = []
  for samples in batch:
    #print(samples)
    lst.append(torch.tensor(samples[key]))
    #torch.cat([x.float() for x in lst], dim=1) 
  return lst 

In [300]:

def custom_collate(batch):
  return {"input_ids":nn.utils.rnn.pad_sequence(stack_vals(batch,"input_ids"),batch_first=True), "attention_mask":nn.utils.rnn.pad_sequence(stack_vals(batch,"attention_mask"),batch_first=True), "label":torch.tensor(stack_vals(batch,"label"))}

train_df = data[data.data_split=="train"]
train_ds = Dataset(train_df)
#train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,collate_fn=custom_collate)
train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

val_df = data[data.data_split=="val"]
val_ds = Dataset(val_df)
#val_ldr = torch.utils.data.DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True,collate_fn=custom_collate)
val_ldr = torch.utils.data.DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True)

In [301]:
class LSTMClassifier(nn.Module):
  def __init__(self, batch_size, output_size, hidden_size, bert):
    super(LSTMClassifier, self).__init__()

    self.bert = bert
    self.batch_size = batch_size
    self.output_size = output_size
    self.hidden_size = hidden_size
    
   # embedding_dim = bert.config.to_dict()['hidden_dim']
    embedding_dim = 768
    self.linear = nn.Linear(embedding_dim, hidden_size)
    self.relu = nn.ReLU()
    #self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size= hidden_size)
    #self.linear2 = nn.Linear(hidden_size, 512)
    self.label = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()
    
  def forward(self, input_ids, attention_mask, batch_size=None):
    input_ids.squeeze_(0)
    attention_mask.squeeze_(0)
    
    # batch_size, len_seq, len_chunk = input_ids.shape 
    # input_ids = input_ids.reshape(-1, input_ids.shape[-1]) #batch_size,num_choices
    # attention_mask = attention_mask.reshape(-1, attention_mask.shape[-1])

    
  
    
    #input_ids = input_ids.permute(0, 2, 1)
    #attention_mask = attention_mask.permute(0, 2, 1)
    # print("input", input_ids.dtype, input_ids.shape)
    # print("input", attention_mask.dtype, input_ids.shape)
    with torch.no_grad():
      embedded = self.bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]

    # input = embedded.reshape(len_seq * len_chunk, batch_size, -1)
    #print("e", embedded.shape,embedded.dtype)
    #print(input)
    

    #input = embedded.permute(1, 0, 2)
    #input = input.reshape(-1, 1,  input.shape[-1])
    
    input = torch.mean(embedded,dim=0)
    input = torch.mean(input,dim=0)
    #print(input.shape)
    #print("lstm", input.shape,input.dtype)
    
    linear_output = self.linear(input)
    #linear_output = self.relu(linear_output)
    #linear_output = self.linear2(linear_output)
    #print(linear_output.shape)
    
    
    #final_output = self.sigmoid(self.label(linear_output))
    final_output = self.label(linear_output)
    
    
    #print(final_output)

    #output, (final_hidden_state, final_cell_state) = self.lstm(input) #lstm is (seq_len, batch, input_size)
    #final_output = self.label(final_hidden_state[-1]) # final_hidden_state.size() = (1, batch_size, hidden_size) & final_output.size() = (batch_size, output_size)
    #print("label", final_output.shape,final_output.dtype)
    
    return final_output.unsqueeze_(0), linear_output
    #return final_output, linear_output

In [302]:

model = LSTMClassifier(batch_size=BATCH_SIZE, output_size=OUTPUT_DIM, hidden_size=HIDDEN_DIM, bert=bert)
                    

In [303]:
torch.cuda.is_available()

True

In [305]:
import torch.optim as optim

learning_rate = 3e-2
weight_decay = 1e-5
eps = 1e-6
optimizer = optim.AdamW(model.parameters(),lr=learning_rate,
                              eps=eps, weight_decay=weight_decay)
#criterion = nn.L1Loss()
criterion = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #edit -> notebook settings to change 
#model.load_state_dict(torch.load("/content/gdrive/MyDrive/6864_project/model_best.pt", map_location="cuda")) 
model = model.to(device)
criterion = criterion.to(device)


In [293]:

def train(model, iterator, optimizer, criterion):
    best_valid_loss = float('inf')
    epoch_loss = 0
    
    model.train()
    
    step_id = 0
    for batch in iterator:
        
        optimizer.zero_grad()
        #print(batch["input_ids"])
        input_ids=batch["input_ids"]
        attention_mask=batch["attention_mask"]
        # input_ids.squeeze_(0)
        # attention_mask.squeeze_(0)
        #embed = bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]
        predictions, embedding = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
        #print("ll", batch["label"].shape, predictions.shape)
        loss = criterion(predictions.to(device), batch["label"].to(device))
        
        loss.backward()
        
        optimizer.step()
        
        item_loss = loss.item()
        epoch_loss += item_loss
        
        if item_loss > 800:
          print(item_loss, input_ids.shape)
        if step_id %  100 == 1:
            print(f'At step {step_id}, the loss = {item_loss}, {epoch_loss / step_id}', predictions,embedding)
        # if step_id % 4000 == 1 and step_id != 1:
        #     valid_loss  = evaluate(model, val_ldr, criterion)
        #     model.train()
        #     torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_best_2.pt')
        #     print(f'saved model with valid loss {valid_loss}')
        step_id += 1

    torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_epoch_upvote_final.pt') 

    outputs = []



      
    step_id = 0
    with torch.no_grad():
        
      for batch in train_ldr:
          
          #optimizer.zero_grad()
          #print(batch["input_ids"])
          input_ids=batch["input_ids"]
          #print(batch)
          attention_mask=batch["attention_mask"]
          # input_ids.squeeze_(0)
          # attention_mask.squeeze_(0)
          #embed = bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]
          predictions, embed = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
          #print("ll", batch["label"].shape, predictions.shape)
          loss = criterion(predictions.to(device), batch["label"].to(device))
          
          # loss.backward()
          
          # optimizer.step()
          
          # item_loss = loss.item()
          # epoch_loss += item_loss
          
          # if item_loss > 800:
          #   print(item_loss, input_ids.shape)
          # if step_id %  100 == 1:
          #     print(f'At step {step_id}, the loss = {item_loss}, {epoch_loss / step_id}')
          # if step_id % 4000 == 1 and step_id != 1:
          #     valid_loss  = evaluate(model, val_ldr, criterion)
          #     model.train()
          #     torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_best_2.pt')
          #     print(f'saved model with valid loss {valid_loss}')
          step_id += 1
          
          #latest_output = {"id": batch["id"][0],"label": batch["label"][0].numpy(),"prediction":torch.argmax(predictions[0]).detach().cpu().numpy(),"embedding":embed.detach().cpu().numpy() }
          latest_output = {"id": batch["id"][0],"label": batch["label"][0].numpy(), "prediction":predictions[0].detach().cpu().numpy(),"embedding":embed.detach().cpu().numpy() }
          outputs.append(latest_output)
          print(latest_output)
    return epoch_loss / len(iterator), 

In [294]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    
    model.eval()
    
    step_id = 0
    with torch.no_grad():
    
        for batch in iterator:

          input_ids=batch["input_ids"]
          attention_mask=batch["attention_mask"]
          # input_ids.squeeze_(0)
          # attention_mask.squeeze_(0)
          #embed = bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]
          predictions = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))[0]
          
          loss = criterion(predictions, batch["label"].to(device))
            
          item = loss.item()
          #print(item)
          epoch_loss += item
          if step_id %  50 == 1:
            print(f'At step {step_id}, the loss = {epoch_loss / step_id}', predictions)
          step_id += 1

    return epoch_loss / len(iterator)

In [295]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [274]:
best_valid_loss = float('inf')

for epoch in range(30):
    
    start_time = time.time()
    
    # if epoch == 0:
    #   valid_loss  = evaluate(model, val_ldr, criterion)
      # print(f'\t Val. Loss: {valid_loss}')
    train_loss  = train(model, train_ldr, optimizer, criterion)
    #valid_loss  = evaluate(model, val_ldr, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_best_epoch_bin_upvote_latest_40.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss}')
    print(f'\t Val. Loss: {valid_loss}')

At step 1, the loss = 0.19769155979156494, 0.7897641658782959 tensor([1.0677], device='cuda:0', grad_fn=<AddBackward0>)
At step 101, the loss = 0.030549347400665283, 0.13697821373986727 tensor([0.9305], device='cuda:0', grad_fn=<AddBackward0>)
At step 201, the loss = 0.018525660037994385, 0.12993871542944838 tensor([0.9815], device='cuda:0', grad_fn=<AddBackward0>)
At step 301, the loss = 0.2634800672531128, 0.11566106960226927 tensor([0.9035], device='cuda:0', grad_fn=<AddBackward0>)
At step 401, the loss = 0.05535328388214111, 0.116011131657032 tensor([0.9446], device='cuda:0', grad_fn=<AddBackward0>)
At step 501, the loss = 0.08559703826904297, 0.11531420114511501 tensor([0.9144], device='cuda:0', grad_fn=<AddBackward0>)
At step 601, the loss = 0.02779257297515869, 0.11562993963823938 tensor([0.9478], device='cuda:0', grad_fn=<AddBackward0>)
At step 701, the loss = 0.11831986904144287, 0.11878987815172629 tensor([1.0183], device='cuda:0', grad_fn=<AddBackward0>)
At step 801, the los

KeyboardInterrupt: ignored

In [307]:
best_valid_loss = float('inf')
epoch_loss = 0

model.train()

step_id = 0
for batch in train_ldr:
    
    optimizer.zero_grad()
    #print(batch["input_ids"])
    input_ids=batch["input_ids"]
    attention_mask=batch["attention_mask"]
    # input_ids.squeeze_(0)
    # attention_mask.squeeze_(0)
    #embed = bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]
    predictions, embedding = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
    #print("ll", batch["label"].shape, predictions.shape)
    loss = criterion(predictions.to(device), batch["label"].to(device))
    
    loss.backward()
    
    optimizer.step()
    
    item_loss = loss.item()
    epoch_loss += item_loss
    
    if item_loss > 800:
      print(item_loss, input_ids.shape)
    if step_id %  100 == 1:
        print(f'At step {step_id}, the loss = {item_loss}, {epoch_loss / step_id}', torch.argmax(predictions),embedding)
    # if step_id % 4000 == 1 and step_id != 1:
    #     valid_loss  = evaluate(model, val_ldr, criterion)
    #     model.train()
    #     torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_best_2.pt')
    #     print(f'saved model with valid loss {valid_loss}')
    step_id += 1

torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_epoch_comment_bin_final.pt') 



# outputs = []

# step_id = 0
# with torch.no_grad():
    
#   for batch in train_ldr:
      
#       #optimizer.zero_grad()
#       #print(batch["input_ids"])
#       input_ids=batch["input_ids"]
#       #print(batch)
#       attention_mask=batch["attention_mask"]
#       # input_ids.squeeze_(0)
#       # attention_mask.squeeze_(0)
#       #embed = bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]
#       predictions, embed = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
#       #print("ll", batch["label"].shape, predictions.shape)
#       loss = criterion(predictions.to(device), batch["label"].to(device))
      
#       # loss.backward()
      
#       # optimizer.step()
      
#       # item_loss = loss.item()
#       # epoch_loss += item_loss
      
#       # if item_loss > 800:
#       #   print(item_loss, input_ids.shape)
#       # if step_id %  100 == 1:
#       #     print(f'At step {step_id}, the loss = {item_loss}, {epoch_loss / step_id}')
#       # if step_id % 4000 == 1 and step_id != 1:
#       #     valid_loss  = evaluate(model, val_ldr, criterion)
#       #     model.train()
#       #     torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_best_2.pt')
#       #     print(f'saved model with valid loss {valid_loss}')
#       step_id += 1
      
#       latest_output = {"id": batch["id"][0],"label": batch["label"][0].numpy(),"prediction":torch.argmax(predictions[0]).detach().cpu().numpy(),"embedding":embed.detach().cpu().numpy() }
#       #latest_output = {"id": batch["id"][0],"label": batch["label"][0].numpy(), "prediction":predictions[0].detach().cpu().numpy(),"embedding":embed.detach().cpu().numpy() }
#       outputs.append(latest_output)
#       print(latest_output)

Streaming output truncated to the last 5000 lines.
      dtype=float32)}
{'id': '402ah5', 'label': array(3), 'prediction': array(2), 'embedding': array([ 8.415067  , -4.753512  , 12.143201  ,  2.134688  ,  6.4654098 ,
        8.9415455 , -5.672551  , 17.762812  , -3.1842036 , -5.367858  ,
       -3.6263082 ,  4.8453984 ,  8.086041  , 13.685172  ,  1.97813   ,
       -0.53025985,  5.5113297 ,  0.16117556, -9.986987  , -1.8207893 ,
        3.997376  ,  4.4984875 , -3.5497754 ,  3.8046787 ,  5.9706497 ,
        2.76082   ,  4.0061383 ,  1.4753298 , -1.6050515 ,  2.6315794 ,
       17.896452  , -0.039607  , -3.709329  , 13.988048  ,  3.3995926 ,
        4.823241  , -3.4515543 , -4.922634  , -5.584054  , -9.880497  ],
      dtype=float32)}
{'id': 'hm9f4i', 'label': array(4), 'prediction': array(0), 'embedding': array([  5.452881  ,  -9.190215  ,   1.8579181 ,   2.6770396 ,
         6.286623  ,   8.9273615 ,  -4.577173  ,   9.632785  ,
         0.06029493,  -2.2210407 ,  -5.877441  ,   6.584

KeyboardInterrupt: ignored

In [309]:
outputs = []

step_id = 0
with torch.no_grad():
    
  for i, batch in enumerate(train_ldr):
      
      #optimizer.zero_grad()
      #print(batch["input_ids"])
      input_ids=batch["input_ids"]
      #print(batch)
      attention_mask=batch["attention_mask"]
      # input_ids.squeeze_(0)
      # attention_mask.squeeze_(0)
      #embed = bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]
      predictions, embed = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
      #print("ll", batch["label"].shape, predictions.shape)
      loss = criterion(predictions.to(device), batch["label"].to(device))
      
      # loss.backward()
      
      # optimizer.step()
      
      # item_loss = loss.item()
      # epoch_loss += item_loss
      
      # if item_loss > 800:
      #   print(item_loss, input_ids.shape)
      # if step_id %  100 == 1:
      #     print(f'At step {step_id}, the loss = {item_loss}, {epoch_loss / step_id}')
      # if step_id % 4000 == 1 and step_id != 1:
      #     valid_loss  = evaluate(model, val_ldr, criterion)
      #     model.train()
      #     torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_best_2.pt')
      #     print(f'saved model with valid loss {valid_loss}')
      step_id += 1
      
      latest_output = {"id": batch["id"][0],"label": batch["label"][0].numpy(),"prediction":torch.argmax(predictions[0]).detach().cpu().numpy(),"embedding":embed.detach().cpu().numpy() }
      #latest_output = {"id": batch["id"][0],"label": batch["label"][0].numpy(), "prediction":predictions[0].detach().cpu().numpy(),"embedding":embed.detach().cpu().numpy() }
      outputs.append(latest_output)
      if i % 100 == 1:
        print(i, latest_output)

1 {'id': '2ya945', 'label': array(3), 'prediction': array(0), 'embedding': array([  4.1602316 ,  -5.2353797 ,   3.6112597 ,  -1.470049  ,
         7.03216   ,   6.791121  ,  -7.1040945 ,   7.635002  ,
         4.4856257 ,   0.8902213 ,   5.829204  ,  10.28657   ,
         3.432365  ,  -0.01969727,  -3.3254535 ,  -0.5549062 ,
         8.990015  ,   0.5622908 ,  -6.708647  ,  -5.968016  ,
        -1.4742253 ,  -5.404928  , -11.123014  ,  -0.35820445,
         0.9242945 ,   4.540387  ,   3.380693  ,  -2.4720714 ,
         1.5492196 ,   6.231711  ,  -1.3721606 ,   2.2957785 ,
        -6.5063577 ,  -5.839273  ,  -3.3089085 ,   1.8956743 ,
        -2.5156324 ,  -6.4011273 , -10.311403  ,  -2.405053  ],
      dtype=float32)}
101 {'id': '5b7ko9', 'label': array(1), 'prediction': array(2), 'embedding': array([  4.434273  ,  -1.4615256 ,   3.1104107 ,  -3.060845  ,
         8.891187  ,  13.890363  ,  -0.15677974,   8.740213  ,
        -6.4617696 ,  -3.6407723 ,   4.4819903 ,   3.8247259 ,
      

In [277]:
#model.load_state_dict(torch.load("/content/gdrive/MyDrive/6864_project/model_best_epoch_topic_model_10.pt", map_location="cuda")) 
#model.load_state_dict(torch.load("/content/gdrive/MyDrive/6864_project/model_best_epoch_topic_model_10_hiddenl.pt", map_location="cuda")) 

#model.load_state_dict(torch.load("/content/gdrive/MyDrive/6864_project/model_best_epoch_bin_comments_40.pt", map_location="cuda")) 

#model.load_state_dict(torch.load('/content/gdrive/MyDrive/6864_project/model_best_epoch_bin_upvote_latest_40.pt', map_location="cuda")) 

#model = model.to(device)
outputs = []
step_id = 0 

model.train()
    
step_id = 0
with torch.no_grad():
    
  for batch in train_ldr:
      
      #optimizer.zero_grad()
      #print(batch["input_ids"])
      input_ids=batch["input_ids"]
      #print(batch)
      attention_mask=batch["attention_mask"]
      # input_ids.squeeze_(0)
      # attention_mask.squeeze_(0)
      #embed = bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]
      predictions, embed = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
      #print("ll", batch["label"].shape, predictions.shape)
      loss = criterion(predictions.to(device), batch["label"].to(device))
      
      # loss.backward()
      
      # optimizer.step()
      
      # item_loss = loss.item()
      # epoch_loss += item_loss
      
      # if item_loss > 800:
      #   print(item_loss, input_ids.shape)
      # if step_id %  100 == 1:
      #     print(f'At step {step_id}, the loss = {item_loss}, {epoch_loss / step_id}')
      # if step_id % 4000 == 1 and step_id != 1:
      #     valid_loss  = evaluate(model, val_ldr, criterion)
      #     model.train()
      #     torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_best_2.pt')
      #     print(f'saved model with valid loss {valid_loss}')
      step_id += 1
      
      #latest_output = {"id": batch["id"][0],"label": batch["label"][0].numpy(),"prediction":torch.argmax(predictions[0]).detach().cpu().numpy(),"embedding":embed.detach().cpu().numpy() }
      latest_output = {"id": batch["id"][0],"label": batch["label"][0].numpy(), "prediction":predictions[0].detach().cpu().numpy(),"embedding":embed.detach().cpu().numpy() }
      outputs.append(latest_output)
      print(latest_output)

#torch.save(model.state_dict(), '/content/gdrive/MyDrive/6864_project/model_epoch_2.pt') 
#return epoch_loss / len(iterator), 

{'id': '9wapte', 'label': array(1., dtype=float32), 'prediction': array(0.846924, dtype=float32), 'embedding': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)}
{'id': 'ibjhvg', 'label': array(0.77, dtype=float32), 'prediction': array(0.846924, dtype=float32), 'embedding': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)}
{'id': 'g3axwo', 'label': array(1., dtype=float32), 'prediction': array(0.846924, dtype=float32), 'embedding': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)}
{'id': '4o7xl4', 'label': array(0.86, dtype=float32), 'prediction': arr

KeyboardInterrupt: ignored

In [ ]:
epoch_loss = 0
    
model.eval()

step_id = 0
with torch.no_grad():

    for batch in val_ldr:

      input_ids=batch["input_ids"]
      attention_mask=batch["attention_mask"]
      # input_ids.squeeze_(0)
      # attention_mask.squeeze_(0)
      #embed = bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"]
      predictions = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))[0]
      
      loss = criterion(predictions, batch["label"].to(device))
        
      item = loss.item()
      #print(item)
      epoch_loss += item
      if step_id %  50 == 1:
        print(f'At step {step_id}, the loss = {epoch_loss / step_id}', predictions)
      step_id += 1

print(epoch_loss / len(val_ldr))

At step 1, the loss = 24.306707859039307 tensor([[-5.3498,  1.8545,  9.5576, -3.5304, -1.0714, -7.9480]],
       device='cuda:0')
At step 51, the loss = 8.517121967074333 tensor([[1.5074, 2.9704, 6.4397, 4.5005, 6.4628, 7.0417]], device='cuda:0')
At step 101, the loss = 7.835755085276699 tensor([[-0.7845, -1.8028, 10.6877,  0.5637, -3.4279,  2.3172]],
       device='cuda:0')
At step 151, the loss = 7.778232285363532 tensor([[-0.5383,  7.5661,  6.9578, -6.3971,  2.6602, -8.1394]],
       device='cuda:0')
At step 201, the loss = 7.768717945341744 tensor([[-1.1808, -2.0625, 16.4843, -2.0099,  1.4287, -7.6264]],
       device='cuda:0')
At step 251, the loss = 7.818694727505047 tensor([[-0.4755,  4.6801,  5.9745, -6.1105,  2.0905, -3.9361]],
       device='cuda:0')
At step 301, the loss = 7.7933067519518655 tensor([[-2.7460,  3.0547,  7.7337, -3.6187,  6.2494, -5.1797]],
       device='cuda:0')
At step 351, the loss = 7.732156763101732 tensor([[ 5.4527,  4.2206,  6.2023, -1.3368,  3.8165, -

In [ ]:
df = pd.DataFrame(outputs)

In [ ]:
df.embedding[0]

,id,prediction,embedding
0,fkaknf,69.21656,"[19.24438, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,abj1km,58.407047,"[15.710117, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,53j82y,63.696365,"[17.89371, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,5rkvvp,79.430534,"[24.06216, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,hw124d,76.897675,"[21.705954, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...
56,7yf506,52.53534,"[15.077574, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
57,hbch9s,79.65956,"[23.627333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
58,3jagty,79.33392,"[22.734709, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
59,749h4v,81.368385,"[24.27338, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
df.to_csv("bert_upvote.csv")